In [18]:
!pip install wooldridge
import wooldridge as wd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
wage = wd.data('wage1')[['wage', 'educ']]
y = wage['wage']
x = wage['educ']

In [ ]:
plt.scatter(x, y)

## White

1. y en x

In [ ]:
x = np.c_[np.ones(len(x)), x]

In [9]:
betas = np.linalg.inv(x.T.dot(x)).dot(x.T.dot(y))

2. Encontrar el error estimado

In [10]:
y_est = x.dot(betas)
y_est_2 = y_est ** 2
u_est = y - y_est
u_est_2 = u_est ** 2

3. $\hat{u}^2$ sobre $\hat{y}$ y $\hat{y}^2$

In [11]:
x_u = np.c_[np.ones(len(y_est)), y_est, y_est_2]

In [13]:
betas_u = np.linalg.inv(x_u.T.dot(x_u)).dot(x_u.T.dot(u_est_2))

In [14]:
u_est_2_est = x_u.dot(betas_u)
sst = sum((u_est_2 - u_est_2.mean()) ** 2) 
sse = sum((u_est_2_est - u_est_2.mean()) ** 2)
r2 = sse / sst

In [16]:
chi_stat = len(x_u) * r2

In [17]:
chi_stat

23.240554444415054

In [19]:
chi = st.chi2(2)
p_valor = 1 - chi.cdf(chi_stat)

In [20]:
p_valor

8.982096656673377e-06

## Breusch-Pagan

In [21]:
betas_u = np.linalg.inv(x.T.dot(x)).dot(x.T.dot(u_est_2))

In [22]:
betas_u

array([-11.59282616,   1.8278615 ])

In [23]:
u_est_2_est = x.dot(betas_u)
sst = sum((u_est_2 - u_est_2.mean()) ** 2) 
sse = sum((u_est_2_est - u_est_2.mean()) ** 2)
r2 = sse / sst

In [24]:
r2

0.029098153431596895

In [25]:
chi_stat = len(x) * r2

In [26]:
chi_stat

15.305628705019966

In [27]:
chi = st.chi2(1)
p_valor = 1 - chi.cdf(chi_stat)

In [28]:
p_valor

9.144363288382529e-05

In [29]:
import statsmodels.api as sm
import statsmodels.stats.api as sms

In [35]:
x = sm.add_constant(x)
model = sm.OLS(y, x)
reg = model.fit()
wh = sms.het_white(reg.resid, reg.model.exog)

In [36]:
bp = sms.het_breuschpagan(reg.resid, reg.model.exog)

In [43]:
df = pd.DataFrame(np.vstack([np.array(wh),
np.array(bp)]).T, columns=['white', 'breusch-pagan'], index=['chi_stat', 'p_val','f_stat','p_val'])

In [44]:
df

,white,breusch-pagan
chi_stat,23.240554,15.305629
p_val,0.000009,0.000091
f_stat,12.088097,15.704401
p_val,0.000007,0.000084
